In [1]:
#네이버기사 크롤링 엑셀저장
import requests
from bs4 import BeautifulSoup
import time
from openpyxl import Workbook
from tqdm import tqdm
import pandas as pd
import re

In [2]:
start_date='2023.07.01'
end_date='2024.02.22'
date_list=pd.date_range(start=start_date, end=end_date, freq='D').astype(str)
print(date_list)

Index(['2023-07-01', '2023-07-02', '2023-07-03', '2023-07-04', '2023-07-05',
       '2023-07-06', '2023-07-07', '2023-07-08', '2023-07-09', '2023-07-10',
       ...
       '2024-02-13', '2024-02-14', '2024-02-15', '2024-02-16', '2024-02-17',
       '2024-02-18', '2024-02-19', '2024-02-20', '2024-02-21', '2024-02-22'],
      dtype='object', length=237)


In [3]:
#사용자입력
# "검색어를 입력하세요"
keyword = '쿠팡'

#"몇 페이지까지 크롤링 할까요?"
lastpage = 20

In [11]:
#엑셀생성하기
wb = Workbook()

#엑셀 시트 생성하기
ws = wb.create_sheet(keyword)

#열 너비 조절
ws.column_dimensions['A'].width = 60
ws.column_dimensions['B'].width = 60
ws.column_dimensions['C'].width = 120
ws.column_dimensions['D'].width = 60

#행 번호
row = 1

start_date = '2023.07.01'
end_date = '2024.02.19'
#페이지 번호
pageNum = 1
for i in range(1, lastpage*10, 10) :
    print(f"{pageNum}페이지 크롤링중입니다 =================")
    response = requests.get(f'https://search.naver.com/search.naver?where=news&query={keyword}&start={i}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}')
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.select("div.info_group") #뉴스기사 div 10개 추출

    for article in articles:
        try:
            links = article.select("a.info") #리스트
            media = article.find_all('a')[0].text
            if len(links) >= 2: #링크가 2개 이상이면
                url = links[1].attrs['href'] #두번째 링크의 href를 추출
                response = requests.get(url, headers={'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})
                html = response.text
                soup_sub = BeautifulSoup(html, 'html.parser')

                #만약 연예 뉴스라면
                if "entertain" in response.url:
                    title = soup_sub.select_one(".end_tit")
                    content = soup_sub.select_one("#articeBody")
                    date = soup_sub.select_one("div.article_info > span > em")
                #만약 스포츠 뉴스라면
                elif "sports" in response.url:
                    title = soup_sub.select_one("h4.title")
                    content = soup_sub.select_one("#newsEndContents")
                    #본문 내용안에 불필요한 div, p제거
                    divs = content.select("div")
                    for div in divs:
                        div.decompose()
                    paragraphs = content.select("p")
                    for p in paragraphs:
                        p.decompose()
                else:
                    title = soup_sub.select_one(".media_end_head_headline")
                    content = soup_sub.select_one("#newsct_article")
                    date = soup_sub.select_one("span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME")

                print("=======링크======= \n", url)
                print("=======제목======= \n", title.text.strip())
                print("=======본문======= \n", content.text.strip())
                print("=======날짜======= \n", date)
                ws['A1'] = 'URL'
                ws['B1'] = '기사제목'
                ws['C1'] = '기사본문'
                ws['D1'] = '업로드날짜'
                ws['E1'] = '언론사'

                ws[f'A{row}'] = url
                ws[f'B{row}'] = title.text.strip()
                ws[f'C{row}'] = content.text.strip()
                ws[f'D{row}'] = date.text.strip()
                ws[f'E{row}'] = media.strip()

                row=row+1
        except:
            pass
    pageNum = pageNum+1
wb.save(f'{keyword}_result.date_240219.xlsx')

1페이지 크롤링중입니다 =================
2페이지 크롤링중입니다 =================
3페이지 크롤링중입니다 =================
4페이지 크롤링중입니다 =================
5페이지 크롤링중입니다 =================
6페이지 크롤링중입니다 =================
7페이지 크롤링중입니다 =================
8페이지 크롤링중입니다 =================
9페이지 크롤링중입니다 =================
10페이지 크롤링중입니다 =================
11페이지 크롤링중입니다 =================
12페이지 크롤링중입니다 =================
13페이지 크롤링중입니다 =================
14페이지 크롤링중입니다 =================
15페이지 크롤링중입니다 =================
16페이지 크롤링중입니다 =================
17페이지 크롤링중입니다 =================
18페이지 크롤링중입니다 =================
19페이지 크롤링중입니다 =================
20페이지 크롤링중입니다 =================
